# Imports

In [ ]:
import datetime
import os
import re
import pandas as pd

from dotenv import load_dotenv
from matplotlib import pyplot as plt
from notion_client import Client

from IPython.display import display, HTML

# Objects and functions

In [ ]:
def get_domain(url):
    return re.search(r"https?://(?:www\.)?([^/]+)", url).group(1)

In [ ]:
def set_old_status(df_offers):
    df_offers["Status"] = df_offers.apply(
        lambda row: "old" if row["Status"] == "active" and pd.to_datetime(row["Created Time"]).date() < THIRTY_DAYS_AGO else row["Status"],
        axis=1
    )
    return df_offers

In [ ]:
def day_list_generator(start_date, end_date):
    days = []
    current_date = start_date
    while current_date <= end_date:
        days.append(current_date)
        current_date += datetime.timedelta(days=1)
    return days

# Prepare

## Load environment variables

In [ ]:
load_dotenv()

THIRTY_DAYS_AGO = datetime.datetime.utcnow().date() - datetime.timedelta(days=30)
NOTION_TOKEN = os.getenv("NOTION_ID")
NOTION_DB_ID = os.getenv("NOTION_DB_ID")
OLD_DAYS_COUNT = datetime.datetime.utcnow() - datetime.timedelta(
    days=int(os.getenv("OLD_DAYS_COUNT"))
)

## Load data from notion

In [ ]:
client = Client(auth=NOTION_TOKEN)
notion_database = client.databases.query(database_id=NOTION_DB_ID)

In [ ]:
notion_database

## Dataframe preparation

In [ ]:
try:
    data = [
        {
            "Name": offer["properties"]["Offer name"]["title"][0]["plain_text"],
            "Created Time": offer["created_time"],
            "Last Edited Time": offer["last_edited_time"],
            "Status": offer["properties"]["Status"]["select"]["name"],
            "Company": offer["properties"]["Company"]["select"]["name"],
            "Link to Offer": offer["properties"]["Link to offer"]["url"],
            "Link to Note": offer["url"],
        }
        for offer in notion_database["results"]
    ]
except TypeError as e:
    if str(e) == "'NoneType' object is not subscriptable":
        print("Probably some offer don't have assigned status")
    else:
        print("Unknown Type Error", e)
except Exception as e:
    print(e)

In [ ]:
df_offers = pd.DataFrame(data)

In [ ]:
df_offers.head()

In [ ]:
df_offers["Last Edited Time"] = pd.to_datetime(
    df_offers["Last Edited Time"]
).dt.strftime("%Y-%m-%d %H:%M:%S")
df_offers["Created Time"] = pd.to_datetime(df_offers["Created Time"]).dt.strftime(
    "%Y-%m-%d %H:%M:%S"
)

In [ ]:
df_offers["Status"] = df_offers["Status"].astype("string")
df_offers["Company"] = df_offers["Company"].astype("string")

In [ ]:
df_offers["Name"] = df_offers.apply(
    lambda offer: (
        f'<a href="{offer["Link to Note"]}" target="_blank">{offer["Name"]}</a>'
        if pd.notnull(offer["Link to Note"])
        else offer["Name"]
    ),
    axis=1,
)

df_offers["Link to Offer"] = df_offers.apply(
    lambda offer: (
        f'<a href="{offer["Link to Offer"]}" target="_blank">{get_domain( offer["Link to Offer"])}</a>'
        if pd.notnull(offer["Link to Offer"]) and offer["Link to Offer"] != "<NA>"
        else "No url"
    ),
    axis=1,
)

df_offers = df_offers.drop("Link to Note", axis=1)

In [ ]:
df_offers = set_old_status(df_offers)

# Analyze

## Show all offers

In [ ]:
display(
    HTML(
        df_offers
        .sort_values(by="Last Edited Time", ascending=False)
        .reset_index(drop=True)
        .to_html(escape=False)
    )
)

## Status stats

In [ ]:
count_stats = df_offers.groupby(by="Status").count()
count_stats = count_stats.rename(columns={"Name": "Offer Count"})
pd.DataFrame(count_stats["Offer Count"])

In [ ]:
count_stats["Offer Count"].plot(
    kind="pie", figsize=(5, 5), title="Status Count", autopct="%.0f%%"
)
plt.ylabel("")
plt.show()

In [ ]:
date_counts = df_offers["Created Time"].apply(lambda x: pd.to_datetime(x).date()).value_counts()
day_list = day_list_generator(min(date_counts.keys()), max(date_counts.keys()))

In [ ]:
plt.figure(figsize=(15, 6))
bars = plt.bar(date_counts.keys(), date_counts.values)
plt.xlabel("Date")
plt.ylabel("Number of Offers")
plt.title("Number of Offers Created per Day")
plt.xticks(day_list[::-2], rotation=45, ha='right')
plt.axvline(x=THIRTY_DAYS_AGO, color='red', linestyle='--', linewidth=2)
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, int(yval), va='bottom', ha='center')

plt.tight_layout()
plt.show()